# Read Data file


In [1]:
file = open("E:/1_DS_course/deep learning/L15/sherlock.txt", encoding="utf-8")
A = file.read()
len(A)

581887

# Preprocessing


In [2]:
Q = []
from re import sub
for i in A[0:10000].split(" "):
    Q.append(sub("[^A-Z0-9]","",i.upper()))
Q[0:10]

['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN']

# Arrange data in inp and op formats


In [3]:
inp = []
op = []
for i in range(4,len(Q),1):
    w0 = Q[i-4]
    w1 = Q[i-3]
    w2 = Q[i-2]
    w3 = Q[i-1]
    w4 = Q[i]
    inp.append(w0 + " " + w1 + " " + w2 + " " + w3)
    op.append(w4)  
import pandas as pd
W = pd.DataFrame([inp,op]).T
W.columns=["Input","Output"]
W.head()

,Input,Output
0,PROJECT GUTENBERGS THE ADVENTURES,OF
1,GUTENBERGS THE ADVENTURES OF,SHERLOCK
2,THE ADVENTURES OF SHERLOCK,HOLMES
3,ADVENTURES OF SHERLOCK HOLMES,BY
4,OF SHERLOCK HOLMES BY,ARTHUR


In [4]:
W.shape

(1683, 2)

# Find uniques


In [5]:
from numpy import unique
uniques = unique(Q)
len(uniques)

768

# Find lookup dictionary for uniques


In [6]:
uniques[1]

'1661LAST'

In [7]:
UL = {}
for i in range(0,len(uniques),1):
    UL[uniques[i]]=i

In [8]:
UL["AND"]

41

In [9]:
UL["SHERLOCK"]

585

# Cube declaration


In [10]:
no_lines = 1683
no_uniques = 768
seq = 4

In [11]:
from numpy import zeros
Xprov = zeros((no_lines,no_uniques,seq))

# Prepare Cube


In [12]:
for line_number,inp_seq in list(enumerate(inp)):
    for word_number,word in enumerate(inp_seq.split(" ")):
        Xprov[line_number,UL[word],word_number]=1

In [13]:
#Xprov[0,525,0]

# Prepare Y


In [14]:
Yprov = zeros((no_lines,no_uniques))
for wordno,word in enumerate(op):
    Yprov[wordno,UL[word]]=1
Yprov

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
UL["OF"]

466

In [16]:
Yprov[0,466]

1.0

# Create a NN


In [17]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.activations import softmax
nn = Sequential()
nn.add(LSTM(128,input_shape=(768,4)))
nn.add(Dense(768))
nn.add(Dense(768,activation='softmax'))
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model = nn.fit(Xprov,Yprov,validation_split=0.1,epochs=3,batch_size=21)

Epoch 1/3
73/73 [==============================] - 24s 307ms/step - loss: 6.3297 - accuracy: 0.0469 - val_loss: 6.4332 - val_accuracy: 0.0651
Epoch 2/3
73/73 [==============================] - 21s 293ms/step - loss: 5.9088 - accuracy: 0.0482 - val_loss: 6.7726 - val_accuracy: 0.0651
Epoch 3/3
73/73 [==============================] - 22s 298ms/step - loss: 5.8239 - accuracy: 0.0462 - val_loss: 7.0798 - val_accuracy: 0.0651


# Prepare input for prediction


In [18]:
I = input("Enter words: ")
I = I.upper()

Enter words: The Adventures of Sherlock


In [24]:
import numpy as np

In [25]:
Xop = zeros((1,no_uniques,seq))
for word_number,word in enumerate(I.split(" ")):
    Xop[0,UL[word],word_number]=1

R = {}
for i in range(0,len(UL.values())):
    R[list(UL.values())[i]]=list(UL.keys())[i]

predicted_val = R[np.argmax(nn.predict(Xop))]
predicted_val


1/1 [==============================] - 0s 37ms/step


'THE'